In [1]:
import json
import os
import math
import librosa

In [ ]:
DATASET_PATH = "C://Users//miran//.vscode//musicClassification//Data//genres_original"
JSON_PATH = "data_map.json"
SAMPLE_RATE = 22050
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

# Data preprocessing and labeling

In [ ]:
def get_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, blocksize=512, num_segments=5):
   # dictionary to store mapping, labels, and MFCCs
    data_map = {
        "mapping": [],
        "labels": [],
        "mfcc": []
    }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / blocksize)

    for i, (root, dirs, files) in enumerate(os.walk(dataset_path)):

      if root is dataset_path:
        continue

      # get genres of music
      genre_label = root.split("/")[-1]
      data_map["mapping"].append(genre_label)
      
      for file in files:

        file_path = os.path.join(dirs, file)
        audio_signal, sf = librosa.load(file_path, sr=SAMPLE_RATE)
        
        for seg in range(num_segments):
          start_seg = samples_per_segment * seg
          end_seg = start_seg + samples_per_segment

          mfcc = librosa.feature.mfcc(audio_signal[start_seg:end_seg], sf, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=blocksize)
          mfcc = mfcc.T

          if len(mfcc) == num_mfcc_vectors_per_segment:
                data_map["mfcc"].append(mfcc.tolist())
                data_map["labels"].append(i-1)
                print("{}, segment:{}".format(file_path, d+1))

    # save mfcc to json
    with open(json_path, "w") as fp:
      json.dump(data_map, fp, indent=4)
